## Working with text data
http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

The goal of this guide is to explore some of the main scikit-learn tools on a single practical task: analyzing a collection of text documents (newsgroups posts) on twenty different topics.

In this section we will see how to:

load the file contents and the categories
extract feature vectors suitable for machine learning
train a linear model to perform categorization
use a grid search strategy to find a good configuration of both the feature extraction components and the classifier




# Loading the 20 newsgroups dataset¶


n the following we will use the built-in dataset loader for 20 newsgroups from scikit-learn. Alternatively, it is possible to download the dataset manually from the website and use the sklearn.datasets.load_files function by pointing it to the 20news-bydate-train sub-folder of the uncompressed archive folder.

In order to get faster execution times for this first example we will work on a partial dataset with only 4 categories out of the 20 available in the dataset:

In [1]:
#ls data/twenty_newsgroups/20news-bydate-train

In [ ]:
#cd data/twenty_newsgroups/

In [ ]:
# %%time
# ! python fetch_data.py

In [2]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.datasets import load_files
import pandas as pd
import numpy as np

In [3]:
categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']

In [ ]:
# train_data_folder = "data/twenty_newsgroups/20news-bydate-train/"
# twenty_train = load_files(train_data_folder)

In [4]:
# test_data_folder = "data/twenty_newsgroups/20news-bydate-test/"
# twenty_test = load_files(test_data_folder)

In [ ]:
# %%time
# from sklearn.datasets import fetch_20newsgroups
# twenty_train = fetch_20newsgroups(subset='train',
#     categories=categories, shuffle=True, random_state=42)

In [5]:
%%time
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',
    categories=categories, shuffle=True, random_state=42)

CPU times: user 203 ms, sys: 281 ms, total: 484 ms
Wall time: 468 ms


In [6]:
twenty_train.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR', 'description'])

In [7]:
twenty_train.DESCR

In [8]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [9]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [10]:
twenty_train.filenames

array(['/home/salas/scikit_learn_data/20news_home/20news-bydate-train/comp.graphics/38440',
       '/home/salas/scikit_learn_data/20news_home/20news-bydate-train/comp.graphics/38479',
       '/home/salas/scikit_learn_data/20news_home/20news-bydate-train/soc.religion.christian/20737',
       ...,
       '/home/salas/scikit_learn_data/20news_home/20news-bydate-train/sci.med/58112',
       '/home/salas/scikit_learn_data/20news_home/20news-bydate-train/sci.med/58578',
       '/home/salas/scikit_learn_data/20news_home/20news-bydate-train/sci.med/58895'],
      dtype='<U92')

The returned dataset is a scikit-learn “bunch”: a simple holder object with fields that can be both accessed as python dict keys or object attributes for convenience, for instance the target_names holds the list of the requested category names:

The files themselves are loaded in memory in the data attribute. For reference the filenames are also available:



In [12]:
len(twenty_train.data)

2257

In [13]:
len(twenty_train.filenames)

2257

print the first line of the firstloaded file:

In [14]:
len(twenty_train.data)

2257

In [15]:
len(twenty_train.filenames)

2257

In [16]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton


In [17]:
print(twenty_train.target_names[twenty_train.target[0]])

comp.graphics


Supervised learning algorithms will require a category label for each document in the training set. In this case the category is the name of the newsgroup which also happens to be the name of the folder holding the individual documents.

For speed and space efficiency reasons scikit-learn loads the target attribute as an array of integers that corresponds to the index of the category name in the target_names list. The category integer id of each sample is stored in the target attribute:

In [18]:
twenty_train.target[:10]

array([1, 1, 3, 3, 3, 3, 3, 2, 2, 2])

Its pissible to get back the gategory names

In [19]:
for t in twenty_train.target[:10]:
    print(twenty_train.target_names[t])

comp.graphics
comp.graphics
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
sci.med
sci.med
sci.med


You might have noticed that the samples were shuffled randomly when we called `fetch_20newsgroups(..., shuffle=True, random_state=42)`: this is useful if you wish to select only a subset of samples to quickly train a model and get a first idea of the results before re-training on the complete dataset later.

# Extracting features from text files
In order to perform ml on text docs, we fitst need to turn the text content into numerical feature vectors


## Bag of words
The most intuitive way to do so is to use a bags of words representation:

1. Assign a fixed integer id to each word occurring in any document of the training set (for instance by building a dictionary from words to integer indices).
2. For each document #i, count the number of occurrences of each word w and store it in X[i, j] as the value of feature #j where j is the index of word w in the dictionary.

The bags of words representation implies that `n_features` is the number of distinct words in the corpus: this number is typically larger than 100,000.

If `n_samples == 10000`, storing `X` as a NumPy array of type float32 would require 10000 x 100000 x 4 bytes = **4GB in RAM** which is barely manageable on today’s computers.

Fortunately, **most values in X will be zeros** since for a given document less than a few thousand distinct words will be used. For this reason we say that bags of words are typically **high-dimensional sparse datasets**. We can save a lot of memory by only storing the non-zero parts of the feature vectors in memory.

**scipy.sparse** matrices are data structures that do exactly this, and `scikit-learn` has built-in support for these structures.

## Tokenizing text with `sklearn`

Text preprocessing, tokenizing and filtering of stopwords are all included in CountVectorizer, which builds a dictionary of features and transforms documents to feature vectors:

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(encoding='utf-8')

In [21]:
X_train_counts = count_vect.fit_transform(twenty_train.data)

In [22]:
X_train_counts.shape

(2257, 35788)

**CountVectorizer** supports counts of N-grams of words or consecutive characters. Once fitted, the vectorizer has built a dictionary of feature indices:



In [23]:
count_vect.vocabulary_.get('algorithm')

4690

The index value of a word in the vocabulary is linked to its frequency in the whole training corpus.


## From occurrences to frequencies
Occurrence count is a good start but there is an issue: longer documents will have higher average count values than shorter documents, even though they might talk about the same topics.

To avoid these potential discrepancies it suffices to divide the number of occurrences of each word in a document by the total number of words in the document: these new features are called tf for Term Frequencies.

Another refinement on top of `tf` is to downscale weights for words that occur in many documents in the corpus and are therefore less informative than those that occur only in a smaller portion of the corpus.

This downscaling is called **tf–idf** for “Term Frequency times Inverse Document Frequency”.

Both `tf` and `tf–idf` can be computed as follows using `TfidfTransformer`:

In [24]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(2257, 35788)

In the above example-code, we 
- firstly use the `fit(..)` method to fit our estimator to the data and 
- secondly the `transform(..)` method to transform our count-matrix to a tf-idf representation. 

These two steps can be combined to achieve the same end result faster by skipping redundant processing. This is done through using the `fit_transform(..)` method as shown below, and as mentioned in the note in the previous section:

In [25]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35788)

## Training a classifier


Now that we have our features, we can train a classifier to try to predict the category of a post. Let’s start with a **naïve Bayes classifier**, which provides a nice baseline for this task. 

scikit-learn includes several variants of this classifier; the one most suitable for word counts is the multinomial variant:

In [26]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

To try to predict the outcome on a new document we need to extract the features using almost the same feature extracting chain as before. 

The difference is that we call `transform` instead of `fit_transform` on the transformers, since they have already been fit to the training set:

In [27]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


## Building a pipeline¶

In order to make the `vectorizer => transformer => classifier` easier to work with, scikit-learn provides a **Pipeline** class that behaves like a compound classifier:

In [28]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])

The names `vect`, `tfidf` and `clf` (classifier) are arbitrary. 

We will use them to perform grid search for suitable hyperparameters below. We can now train the model with a single command:

In [29]:
text_clf.fit(twenty_train.data, twenty_train.target)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

## Evaluation of the performance on the test set
Evaluating the predictive accuracy of the model is equally easy:

In [30]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test',
    categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target) 

0.8348868175765646

We achieved 83.5% accuracy. Let’s see if we can do better with a **linear support vector machine (SVM)**, which is widely regarded as one of the best text classification algorithms (although it’s also a bit slower than naïve Bayes). We can change the learner by simply plugging a different classifier object into our pipeline:

In [31]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, random_state=42,
                                           max_iter=5, tol=None)),
])

text_clf.fit(twenty_train.data, twenty_train.target)  

predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.9127829560585885

We achieved 91.3% accuracy using the SVM. scikit-learn provides further utilities for more detailed performance analysis of the results:

In [32]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,
    target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.95      0.81      0.87       319
         comp.graphics       0.88      0.97      0.92       389
               sci.med       0.94      0.90      0.92       396
soc.religion.christian       0.90      0.95      0.93       398

           avg / total       0.92      0.91      0.91      1502



In [33]:
metrics.confusion_matrix(twenty_test.target, predicted)

array([[258,  11,  15,  35],
       [  4, 379,   3,   3],
       [  5,  33, 355,   3],
       [  5,  10,   4, 379]])

As expected the confusion matrix shows that posts from the newsgroups on atheism and Christianity are more often confused for one another than with computer graphics.

# Parameter tuning using grid search

We’ve already encountered some parameters such as `use_idf` in the `TfidfTransformer`. Classifiers tend to have many parameters as well; e.g., MultinomialNB includes a smoothing parameter `alpha` and `SGDClassifier` has a penalty parameter `alpha` and configurable loss and penalty terms in the objective function (see the module documentation, or use the Python help function to get a description of these).

Instead of tweaking the parameters of the various components of the chain, it is possible to run an exhaustive search of the best parameters on a grid of possible values. We try out all classifiers on either words or bigrams, with or without idf, and with a penalty parameter of either 0.01 or 0.001 for the linear SVM:

In [34]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3),
}

Obviously, such an exhaustive search can be expensive. If we have multiple CPU cores at our disposal, we can tell the grid searcher to try these eight parameter combinations in parallel with the `n_jobs` parameter. If we give this parameter a value of `-1`, grid search will detect how many cores are installed and use them all:

In [35]:
%%time
gs_clf = GridSearchCV(text_clf, parameters, cv=5, iid=False, n_jobs=-1)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 169 µs


The grid search instance behaves like a normal scikit-learn model. Let’s perform the search on a smaller subset of the training data to speed up the computation:

In [36]:
%time gs_clf = gs_clf.fit(twenty_train.data[:400], twenty_train.target[:400])

CPU times: user 1.09 s, sys: 781 ms, total: 1.88 s
Wall time: 5.19 s


The result of calling fit on a GridSearchCV object is a classifier that we can use to predict:



In [37]:
twenty_train.target_names[gs_clf.predict(['God is love'])[0]]

'soc.religion.christian'

The object’s `best_score_` and `best_params_` attributes store the best mean score and the parameters setting corresponding to that score:

In [38]:
gs_clf.best_score_                                  

for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

clf__alpha: 0.001
tfidf__use_idf: True
vect__ngram_range: (1, 2)


A more detailed summary of the search is available at `gs_clf.cv_results_`.

The `cv_results_` parameter can be easily imported into pandas as a DataFrame for further inspection

In [39]:
gs_clf.best_score_                                  

0.9151349867929058

In [40]:
cv_res = gs_clf.cv_results_

In [ ]:
#type(gs_clf.cv_results_)

In [41]:
pd.DataFrame(list(cv_res.items()))

,0,1
0,mean_fit_time,"[0.2750385284423828, 1.3940627574920654, 0.536..."
1,std_fit_time,"[0.08457525955319156, 0.12049648301299191, 0.0..."
2,mean_score_time,"[0.06545276641845703, 0.1236645221710205, 0.05..."
3,std_score_time,"[0.015339086183708918, 0.015709531997860546, 0..."
4,param_clf__alpha,"[0.01, 0.01, 0.01, 0.01, 0.001, 0.001, 0.001, ..."
5,param_tfidf__use_idf,"[True, True, False, False, True, True, False, ..."
6,param_vect__ngram_range,"[(1, 1), (1, 2), (1, 1), (1, 2), (1, 1), (1, 2..."
7,params,"[{'clf__alpha': 0.01, 'tfidf__use_idf': True, ..."
8,split0_test_score,"[0.8658536585365854, 0.8536585365853658, 0.780..."
9,split1_test_score,"[0.8641975308641975, 0.8518518518518519, 0.777..."
